In [1]:
sqlContext.sql("CREATE TEMPORARY TABLE students_table USING com.stratio.datasource.mongodb OPTIONS (host 'mongo:27017', database 'highschool', collection 'students')")
sqlContext.sql("SELECT * FROM students_table where sex='F' and Mjob='teacher' LIMIT 15").collect()

[Row(famsize=u'GT3', traveltime=u'1', guardian=u'mother', Mjob=u'teacher', failures=u'0', studytime=u'2', health=u'2', G3=u'14', address=u'U', internet=u'yes', Pstatus=u'T', higher=u'yes', G2=u'14', sex=u'F', _id=u'574e5fd1f8374e03e66d4f4d', absences=u'2', Dalc=u'1', school=u'GP', goout=u'3', Fjob=u'health', famrel=u'3', age=u'15', paid=u'no', Walc=u'2', schoolsup=u'no', freetime=u'3', G1=u'14', Fedu=u'4', reason=u'reputation', romantic=u'no', activities=u'no', nursery=u'yes', Medu=u'4', famsup=u'yes'),
 Row(famsize=u'GT3', traveltime=u'1', guardian=u'mother', Mjob=u'teacher', failures=u'0', studytime=u'3', health=u'4', G3=u'13', address=u'U', internet=u'yes', Pstatus=u'T', higher=u'yes', G2=u'13', sex=u'F', _id=u'574e5fd1f8374e03e66d4f82', absences=u'0', Dalc=u'2', school=u'GP', goout=u'4', Fjob=u'health', famrel=u'3', age=u'16', paid=u'no', Walc=u'4', schoolsup=u'yes', freetime=u'4', G1=u'14', Fedu=u'3', reason=u'home', romantic=u'no', activities=u'yes', nursery=u'yes', Medu=u'4', fa

In [2]:
df = sqlContext.read.format('com.stratio.datasource.mongodb').options(host='mongo:27017', database='highschool', collection='students').load()

In [3]:
df.filter(df['Mjob'] == 'teacher').take(10)

[Row(famsize=u'GT3', traveltime=u'1', guardian=u'mother', Mjob=u'teacher', failures=u'0', studytime=u'2', health=u'2', G3=u'14', address=u'U', internet=u'yes', Pstatus=u'T', higher=u'yes', G2=u'14', sex=u'F', _id=u'574e5fd1f8374e03e66d4f4d', absences=u'2', Dalc=u'1', school=u'GP', goout=u'3', Fjob=u'health', famrel=u'3', age=u'15', paid=u'no', Walc=u'2', schoolsup=u'no', freetime=u'3', G1=u'14', Fedu=u'4', reason=u'reputation', romantic=u'no', activities=u'no', nursery=u'yes', Medu=u'4', famsup=u'yes'),
 Row(famsize=u'GT3', traveltime=u'2', guardian=u'mother', Mjob=u'teacher', failures=u'0', studytime=u'2', health=u'3', G3=u'13', address=u'U', internet=u'yes', Pstatus=u'T', higher=u'yes', G2=u'12', sex=u'M', _id=u'574e5fd1f8374e03e66d4f50', absences=u'0', Dalc=u'1', school=u'GP', goout=u'3', Fjob=u'other', famrel=u'5', age=u'15', paid=u'no', Walc=u'2', schoolsup=u'no', freetime=u'4', G1=u'12', Fedu=u'3', reason=u'course', romantic=u'no', activities=u'no', nursery=u'yes', Medu=u'4', fam

In [4]:
view = sqlContext.sql("select * from students_table where Fjob='teacher' and romantic='yes'")
view.take(10)

[Row(famsize=u'GT3', traveltime=u'1', guardian=u'mother', Mjob=u'teacher', failures=u'0', studytime=u'2', health=u'5', G3=u'12', address=u'U', internet=u'yes', Pstatus=u'T', higher=u'yes', G2=u'11', sex=u'M', _id=u'574e5fd1f8374e03e66d4f60', absences=u'4', Dalc=u'5', school=u'GP', goout=u'5', Fjob=u'teacher', famrel=u'4', age=u'16', paid=u'yes', Walc=u'5', schoolsup=u'no', freetime=u'4', G1=u'12', Fedu=u'4', reason=u'home', romantic=u'yes', activities=u'yes', nursery=u'yes', Medu=u'4', famsup=u'yes'),
 Row(famsize=u'GT3', traveltime=u'2', guardian=u'mother', Mjob=u'other', failures=u'0', studytime=u'3', health=u'5', G3=u'13', address=u'R', internet=u'yes', Pstatus=u'A', higher=u'yes', G2=u'13', sex=u'M', _id=u'574e5fd1f8374e03e66d4f68', absences=u'4', Dalc=u'1', school=u'GP', goout=u'3', Fjob=u'teacher', famrel=u'2', age=u'16', paid=u'no', Walc=u'1', schoolsup=u'no', freetime=u'4', G1=u'13', Fedu=u'4', reason=u'reputation', romantic=u'yes', activities=u'yes', nursery=u'yes', Medu=u'4',

In [8]:
view.write.format("com.stratio.datasource.mongodb").mode('overwrite').options(host='mongo:27017', database='highschool', collection='studentsview').save()

In [10]:
import pyspark_elastic

In [40]:
import json
def row_to_dict(row):
    d = row.asDict()
    id = d['_id']
    return id, {key: value for key, value in d.iteritems() if key != '_id' and value and key}
view_dicts = view.select("_id", "reason").map(lambda row: row_to_dict)

In [84]:
es_items = df.map(row_to_dict)

In [85]:
sc.setLocalProperty("es.nodes","elasticsearch")
sc.setSystemProperty("es.nodes","elasticsearch")

In [87]:
es_items.saveAsNewAPIHadoopFile(
    path='-', 
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
    keyClass="org.apache.hadoop.io.NullWritable", 
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
    conf={ "es.resource" : "students/items" ,"es.nodes": "elasticsearch"})

In [88]:
es_rdd = sc.newAPIHadoopRDD(
    inputFormatClass="org.elasticsearch.hadoop.mr.EsInputFormat",
    keyClass="org.apache.hadoop.io.NullWritable", 
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable", 
    conf={ "es.resource" : "students/items","es.nodes": "elasticsearch"})

In [89]:
from pyspark.sql import Row
def item_to_row(item):
    id, data = item
    data['_id'] = id
    return Row(**data)
rows = es_rdd.map(item_to_row)

es_df = rows.toDF()
es_df.registerTempTable('es_table')

In [90]:
sqlContext.sql("select * from students_table where Fjob='teacher' and romantic='yes'").collect()

[Row(famsize=u'GT3', traveltime=u'1', guardian=u'mother', Mjob=u'teacher', failures=u'0', studytime=u'2', health=u'5', G3=u'12', address=u'U', internet=u'yes', Pstatus=u'T', higher=u'yes', G2=u'11', sex=u'M', _id=u'574e5fd1f8374e03e66d4f60', absences=u'4', Dalc=u'5', school=u'GP', goout=u'5', Fjob=u'teacher', famrel=u'4', age=u'16', paid=u'yes', Walc=u'5', schoolsup=u'no', freetime=u'4', G1=u'12', Fedu=u'4', reason=u'home', romantic=u'yes', activities=u'yes', nursery=u'yes', Medu=u'4', famsup=u'yes'),
 Row(famsize=u'GT3', traveltime=u'2', guardian=u'mother', Mjob=u'other', failures=u'0', studytime=u'3', health=u'5', G3=u'13', address=u'R', internet=u'yes', Pstatus=u'A', higher=u'yes', G2=u'13', sex=u'M', _id=u'574e5fd1f8374e03e66d4f68', absences=u'4', Dalc=u'1', school=u'GP', goout=u'3', Fjob=u'teacher', famrel=u'2', age=u'16', paid=u'no', Walc=u'1', schoolsup=u'no', freetime=u'4', G1=u'13', Fedu=u'4', reason=u'reputation', romantic=u'yes', activities=u'yes', nursery=u'yes', Medu=u'4',